In [1]:
from ipyparallel import Client
import os
c = Client(profile='mpi')
v = c[:]
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

# Libraries

In [2]:
%%px
%matplotlib inline

import os
import time
import numpy as np
import mpi4py.MPI as MPI

from matplotlib import cm
import matplotlib.pyplot as plt

import triangle

import pandas

from pyBadlands import FVmethod
from pyBadlands import raster2TIN 
from pyBadlands import flowNetwork
from pyBadlands import elevationTIN
from pyBadlands import partitionTIN
from pyBadlands import visualiseTIN
from pyBadlands import visualiseFlow

import warnings
warnings.filterwarnings("ignore")

# Initialise MPI communications
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialisation Phase

In [3]:
%%px
Init_time = time.clock()

## 1. Get DEM regular grid and create Badlands TIN.

In [4]:
#%%px
#if rank==0:
#    help(raster2TIN)

In [5]:
%%px
walltime = time.clock()

recGrid = raster2TIN.raster2TIN('/home/ubuntu/mpiPyLEM/data/regularMR.csv')

tinData = np.column_stack((recGrid.tinMesh['vertices'][:,0],
    recGrid.tinMesh['vertices'][:,1]))

if rank == 0:
    print " - read dataset and perform delaunay triangulation ", time.clock() - walltime

[stdout:3]  - read dataset and perform delaunay triangulation  1.368917


## 2. Partition the TIN

In [6]:
#%%px
#if rank==0:
#    help(partitionTIN.simple)

In [7]:
%%px
walltime = time.clock()

# Set parameters of the finite volume mesh
FVmesh = FVmethod.FVmethod(recGrid.tinMesh['vertices'],
                  recGrid.tinMesh['triangles'],
                  recGrid.tinMesh['edges'])

# Perform partitioning by equivalent domain splitting
RowProc = 4
ColProc = 2
partitionIDs = partitionTIN.simple(tinData[:,0], tinData[:,1], RowProc, ColProc)
FVmesh.partIDs = partitionIDs

# Get each partition global node ID
inGIDs = np.where(partitionIDs == rank)[0]

if rank == 0:
    print " - partition TIN amongst processors ", time.clock() - walltime

[stdout:3]  - partition TIN amongst processors  1.349192


## 3. Build Finite Volume discretisation

In [8]:
#%%px
#if rank==0:
#    help(FVmethod.FVmethod.construct_FV)

In [9]:
%%px

# Define overlapping partitions
allIDs, localTIN = partitionTIN.overlap(tinData[:,0], \
    tinData[:,1], RowProc, ColProc, 2*recGrid.resEdges)

# Set parameters of the finite volume mesh
tMesh = FVmethod.FVmethod(localTIN['vertices'],
                  localTIN['triangles'],
                  localTIN['edges'])

walltime = time.clock()

# Define Finite Volume parameters
totPts = len(tinData[:,0])
FVmesh.neighbours = np.zeros((totPts,20), dtype=np.int) 
FVmesh.neighbours.fill(-2)
FVmesh.edge_lenght = np.zeros((totPts,20), dtype=np.float) 
FVmesh.vor_edges = np.zeros((totPts,20), dtype=np.float) 
FVmesh.control_volumes = np.zeros(totPts, dtype=np.float) 

# Compute Finite Volume parameters
tGIDs, tNgbh, tEdgs, tVors, tVols = tMesh.construct_FV(inGIDs, allIDs, totPts)

FVmesh.neighbours[tGIDs,:tMesh.maxNgbh] = tNgbh
FVmesh.edge_lenght[tGIDs,:tMesh.maxNgbh] = tEdgs
FVmesh.vor_edges[tGIDs,:tMesh.maxNgbh] = tVors
FVmesh.control_volumes[tGIDs] = tVols

if rank == 0:
    print " - FV mesh ", time.clock() - walltime

[stdout:3] 
 - partition TIN including shadow zones  0.154952
 - build the voronoi diagram  0.081921
 - construct Finite Volume representation  0.173406
 - perform MPI communication  0.195217
 - FV mesh  0.77804


## 4. Interpolate elevation

In [10]:
#%%px
#if rank==0:
#    help(elevationTIN.update_border_elevation)

In [11]:
%%px
walltime = time.clock()
inIDs = np.where(FVmesh.partIDs[recGrid.boundsPt:] == rank)[0]
inIDs += recGrid.boundsPt
 
local_elev = np.zeros(totPts)
local_elev.fill(-1.e6)
local_elev[inIDs] = elevationTIN.getElevation(recGrid.regX, recGrid.regY, \
    recGrid.regZ, FVmesh.node_coords[inIDs,:2])
comm.Allreduce(MPI.IN_PLACE, local_elev, op=MPI.MAX)

elevation = elevationTIN.update_border_elevation(local_elev, \
    FVmesh.neighbours, FVmesh.edge_lenght, recGrid.boundsPt, btype='slope')

tinData = np.column_stack((tinData,elevation)) 
if rank == 0:
    print " - interpolate elevation on grid ", time.clock() - walltime

[stdout:3]  - interpolate elevation on grid  0.153686


In [12]:
%%px
if rank == 0:
    print " - Initialisation phase ", time.clock() - Init_time

[stdout:3]  - Initialisation phase  3.894588


In [13]:
%%px
if rank==0:
    print elevation.min()

[stdout:3] -4315.30990895


# Flow computation

In [14]:
%%px
Flow_time = time.clock()

## 1. Perform pit filling

we still use the Planchon & Darboux algorithm but will try the Priority FLood algorithm from Barnes in a near future.

In [15]:
#%%px
#if rank==0:
#    help(elevationTIN.pit_filling_PD)

In [16]:
%%px
walltime = time.clock()
fillH = elevationTIN.pit_filling_PD(elevation, FVmesh.neighbours, \
                                    recGrid.boundsPt, 100. ,0.01)
if rank == 0:
    print " - depression-less algorithm PD with stack", time.clock() - walltime

[stdout:3]  - depression-less algorithm PD with stack 0.711751


## 2. Compute stream network

In [17]:
%%px
walltime = time.clock()
flow = flowNetwork()
ngbhs = FVmesh.neighbours[allIDs,:]
flow.SFD_receivers(fillH, ngbhs, allIDs)
if rank == 0:
    print " - compute receivers parallel ", time.clock() - walltime

# Distribute evenly local minimas to processors
walltime = time.clock()
flow.localbase = np.array_split(flow.base, size)[rank]
flow.ordered_node_array()
if rank == 0:
    print " - compute stack order locally ", time.clock() - walltime
    
walltime = time.clock()
stackNbs = comm.allgather(len(flow.localstack))
globalstack = np.zeros(sum(stackNbs),dtype=flow.localstack.dtype)
comm.Allgatherv(sendbuf=[flow.localstack, MPI.INT], 
             recvbuf=[globalstack, (stackNbs, None), MPI.INT]) 
flow.stack = globalstack
if rank == 0:
    print " - send stack order globally ", time.clock() - walltime

[stdout:3] 
 - compute receivers parallel  0.073697
 - compute stack order locally  0.05384
 - send stack order globally  0.063825


## 3. Compute discharge

In [18]:
%%px
walltime = time.clock()
rain = np.ones(totPts, dtype=float)
rain[:recGrid.boundsPt] = 0.
flow.compute_flow(FVmesh.control_volumes, rain)
if rank == 0:
    print " - compute discharge serial ", time.clock() - walltime

[stdout:3]  - compute discharge serial  0.071271


In [19]:
%%px
if rank == 0:
    print " - Flow computation ", time.clock() - Flow_time

[stdout:3]  - Flow computation  1.025617


# Create Hdf5 outputs

we first define local vertices, triangles and polylines used to create the visualisation outputs.

In [20]:
%%px
out_time = time.clock()
visXlim = np.zeros(2)
visYlim = np.zeros(2)
visXlim[0] = recGrid.rectX.min()
visXlim[1] = recGrid.rectX.max()
visYlim[0] = recGrid.rectY.min()
visYlim[1] = recGrid.rectY.max()

# Done when TIN has been built/rebuilt
outPts, outCells = visualiseTIN.output_cellsIDs(allIDs,inIDs,visXlim,visYlim,
                                        FVmesh.node_coords[:,:2],tMesh.cells)
tcells = np.zeros(size)
tcells[rank] = len(outCells)
comm.Allreduce(MPI.IN_PLACE,tcells,op=MPI.MAX)
tnodes = np.zeros(size)
tnodes[rank] = len(allIDs)
comm.Allreduce(MPI.IN_PLACE,tnodes,op=MPI.MAX)

# Done for every visualisation step
flowIDs, polylines = visualiseFlow.output_Polylines(outPts,flow.receivers[outPts],
                visXlim,visYlim,FVmesh.node_coords[:,:2])
fnodes = np.zeros(size)
fnodes[rank] = len(flowIDs)
comm.Allreduce(MPI.IN_PLACE,fnodes,op=MPI.MAX)
fline = np.zeros(size)
fline[rank] = len(polylines[:,0])
comm.Allreduce(MPI.IN_PLACE,fline,op=MPI.MAX)

we then write to each partition simulation outputs (TIN & flow network) using HdF5.

In [22]:
%%px
step = 0
SimTime = 0.0
folder = 'mpiPyLEM/output'

th5file = 'tin.time'
txmffile = 'tin.time'
txdmffile = 'tin.series.xdmf'

fh5file = 'flow.time'
fxmffile = 'flow.time'
fxdmffile = 'flow.series.xdmf'

# Write HDF5 files
visualiseTIN.write_hdf5(folder,th5file,step,tMesh.node_coords[:,:2],
                        elevation[allIDs],flow.discharge[allIDs],outCells,rank)
visualiseFlow.write_hdf5(folder,fh5file,step,FVmesh.node_coords[flowIDs,:2],elevation[flowIDs],
                        flow.discharge[flowIDs],polylines,rank)

# Combine HDF5 files and write time series
if rank==0:
    visualiseTIN.write_xmf(folder,txmffile,txdmffile,step,SimTime,tcells,tnodes,th5file,size)
    visualiseFlow.write_xmf(folder,fxmffile,fxdmffile,step,SimTime,fline,fnodes,fh5file,size)
    print " - Writing outputs ", time.clock() - out_time

[stdout:3]  - Writing outputs  0.658613
